<a href="https://colab.research.google.com/github/Szozan/2d-strategic-plan/blob/main/deliverables/Step_9_PDF_Transcript_to_Score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script ingests a PDF transcript and has ChatGPT generate scores based on a rubric. If the PDF is too large, this script breaks it up into smaller chunks.

In [1]:
!pip install langchain openai
!pip install langchain-community
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.9/357.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 5.7 MB/s eta 0:00:00


tiktoken can convert text into tokens to transmit to OpenAI

In [20]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.6 MB/s eta 0:00:00


A library to do similarity matching

In [2]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 24.0 MB/s eta 0:00:00


Mount the user's Google Drive as a virtual hard drive so that the PDF of the transcript can be accessed and results saved. This will open up an authorization prompt to all the Colab notebook to access drive. Approve this.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


import necessary libraries

In [4]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
import os
import pandas as pdf

In [21]:
import pandas as pd
import openai
import os
import ast
import time
import numpy as np
import json
from IPython.display import clear_output
from tqdm import tqdm
import textwrap
import tiktoken

Paste in the prompt to be send along with the PDF to ChatGPT. This prompt should include instructions about how the transcript is to be scored on a rubric.

In [39]:
prompt = '''Hi ChatGPT,
You are a researcher observing college-level statistics classrooms to record the activities of instructors and students. Your task is to first understand the coding rubric and then code the provided CSV file, which contains the transcript of a 65-minute college-level introductory statistics class.
Step 1: Review the Codebook
The classroom observation form has separate codes for the instructor and for students. These codes are recorded simultaneously in 2-minute segments based on what the instructor and students are doing. The coding sheet has been uploaded as a PDF for your review.
Instructor Codes:
Lec (Lecturing): The instructor presents course content, with or without visual aids. This includes talking about topics, explaining concepts, going through slides, or any form of direct teaching of course material. Anything the professor said that is vaguely related to the statistics class is lecturing.
RtW (Real-time Writing): The instructor is writing on a chalkboard, whiteboard, overhead projector, etc. Usually coded along with lecturing (Lec).
PQ (Poses a Question): The instructor poses a content-related question to the whole class, an individual student, or a small group. Does not include rhetorical questions or questions the instructor answers themselves.
ExpA (Explains an Answer): The instructor answers/explains a content-related question.
D/V (Demonstration or Video): The instructor shows or conducts a demonstration, video, simulation, or animation explaining or elaborating on course content.
1:1 (One-on-One Exchange): The instructor discusses with one or a few students while not paying attention to the rest of the class.
Mov (Moving): The instructor moves throughout the classroom, pausing to observe and guide student work.
Adm (Administration): The instructor discusses homework, returns tests, reminds students of due dates, etc.
Student Codes:
List (Listening): Students receive content from the instructor through a lecture, video, or demonstration. Usually coded along with lecturing (Lec) or demonstration/video (D/V). Look for keywords indicating passive reception of information. If the teacher is talking, please imagine the students are listening.
AskQ (Asks a Question): A student asks the instructor a question in front of the whole class. Look for keywords such as "question," "ask." Look for question marks (?) or phrases indicating a question is being asked.
AnQ (Answers a Question): A student answers a teacher’s question.
IndW (Individual Work): Students engage in individual thinking or problem-solving.
GrW (Group Work): Students work in groups of 2-6 on a structured assignment, task, project, or other instructor-assigned activity.
SmGD (Small Group Discussion): Students engage in discussion with 2-6 peers about a question posed by the instructor.
WCD (Whole Class Discussion): Students ask and answer questions in front of the whole class while the instructor facilitates.
Coding Instructions:
Capture instructor and student behaviors objectively.
Circle observed behaviors.
If a behavior continues into the next segment, circle the code(s) again.
Use the Notes section for information not included in the codes.
Print the coding sheets and code tables for reference.
Step 2: Instead of sending you to the classroom, I have the transcript of the classroom for you to use to code instructor and student activities in a file. This file that transcribes what has happened in a classroom. Code the Transcript in the file I just uploaded with this message.
Structure:
People: Indicates whether it is teacher speech, student speech, or silence.
Timestamp: Documents the duration in minutes for each speech segment.
Speech: Documents what was said by the teacher or student, or notes silence.
Procedure:
Divide the data into adjacent 2-minute intervals based on the "Timestamp" column. Let's start by creating the 2-minute intervals and proceed with the refined behavior coding. The timestamps needs to add up to 2 which often include multiple rows of data.
Code Behaviors in Each Interval:
For each 2-minute interval, check the "Speech" column to identify if any of the behaviors occurred.
Use specific keywords and patterns to detect behaviors.
Please also use your general knowledge to code behavior if you cannot find specific keywords and patterns to detect behaviors.
Aggregate the Results:
For each 2-minute interval, aggregate the counts of observed behaviors.
Sum up the counts for all intervals to get the total occurrences of each behavior.
Calculate Percentages:
Divide the total counts of each behavior by the total number of intervals to get the percentage of intervals in which each behavior was observed.
Example Calculation:
If out of 30 intervals, 15 intervals show the teacher was lecturing, the score for Lec (Lecturing) is 0.5.
If students asked a question in 3 out of 30 intervals, the score for AskQ (Asks a Question) is 0.1.
If no behavior is observed, the score is zero.
Output: Please provide scores for instructor and student activities for the CVS file I uploaded. Please remember percentages cannot be over 100%.
'''

Check the prompt is correct (since it is hard to read a long horizontal string)

In [40]:
print('\n'.join(textwrap.wrap(prompt, width=50, replace_whitespace=False)))

Hi ChatGPT,
You are a researcher observing
college-level statistics classrooms to record the
activities of instructors and students. Your task
is to first understand the coding rubric and then
code the provided CSV file, which contains the
transcript of a 65-minute college-level
introductory statistics class.
Step 1: Review the
Codebook
The classroom observation form has
separate codes for the instructor and for
students. These codes are recorded simultaneously
in 2-minute segments based on what the instructor
and students are doing. The coding sheet has been
uploaded as a PDF for your review.
Instructor
Codes:
Lec (Lecturing): The instructor presents
course content, with or without visual aids. This
includes talking about topics, explaining
concepts, going through slides, or any form of
direct teaching of course material. Anything the
professor said that is vaguely related to the
statistics class is lecturing. 
RtW (Real-time
Writing): The instructor is writing on a
chalkboard, whiteb

Fetch the PDF transcript from the user's Google Drive. Replace the filepath of the file variable to the correct PDF destination. You can use the Folder on the left side of Colab to navigate through the mounted drive's folders and files. (The folder sample_data can be ignored. That just comes with Colab by default)

In [13]:
file = "/content/drive/MyDrive/ISEA_Test_Audio/sample_classroom_transcript.pdf"
loader = PyPDFLoader(file)
docs = loader.load()

Test that the PDF was loaded in

In [15]:
docs

[Document(metadata={'source': '/content/drive/MyDrive/ISEA_Test_Audio/sample_classroom_transcript.pdf', 'page': 0}, page_content='T: mobs are large crowds of people Usually mobs are not very happy. When they use the word mob they are angry or upset. Another word: gradually. It means that it will happen slowly or taking place. Moving along little by little. S: Like when babies learn to walk T: Yes like babies. They crawl and stand and eventually walk. T: Budge. It means that you gave in. You agree. You give in. T: Summarizes the meaning of the words above. T; We are going to use our inferring poster, which is on the pocket chart. We can use pictures or illustrations to infer. We infer by using our schema. The chart is color coded. T: S1 S1: They were inﬁghting inside the school. T: write I wander why the people behind the little girl were angry. Let’s begin. Hold that though and let us see if the author will answer our question. The title is the Story of Ruby Bridges. I can infer that t

Set up the client connection to OpenAI's API. This requires an API key. Put your API key into the Colab's Secret (left hand menu). Name the secret APIOpenAI.

In [16]:
from google.colab import userdata

# Retrieve the API key from Colab secrets
OPENAI_API_KEY = userdata.get('APIOpenAI')

# Set the API key as an environment variable
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

Split the PDF content into chunks and store them as vectors.

In [47]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 50000, chunk_overlap = 0)
texts = text_splitter.split_documents(docs)

directory = 'index_store'

vector_index = FAISS.from_documents(texts, OpenAIEmbeddings())
vector_index.save_local(directory)

vector_index = FAISS.load_local('index_store', OpenAIEmbeddings(),allow_dangerous_deserialization=True)
retriever = vector_index.as_retriever(search_type = "similarity", search_kwargs = {"k":6})


Send the PDF to Open AI **(There is something wrong at this step. Not sure how to interact with the PDF chunks and how OpenAI expects them to be sent)**

In [48]:
qa_interface = RetrievalQA.from_chain_type(llm = ChatOpenAI(model_name = "gpt-4-turbo-preview"), chain_type = "stuff", retriever = retriever,
                                               return_source_documents = True)

Send the prompt to ChatGPT for rubric scoring

In [53]:
response = qa_interface('Did you get the File?') #Testing
#response = qa_interface(prompt) #Send the rubric scoring prompt

Return the results from ChatGPT

In [54]:
print(response['result'])

I'm sorry, but I can't access or receive files. How can I assist you with your question or topic?




---



Extra code: The following uses the function that Alex created. It may run into issues if we have really large text transcripts.

In [33]:
def send_to_api(prompt):
    """
    This function sends a given prompt to OpenAI's API and retrieves the response.

    Args:
    prompt (str): A string containing the user's input that needs to be processed by the GPT model.

    Returns:
    result: The response from OpenAI's API or an error message if the request fails.
    """

    # Initialize result to None, this will store the API response
    result = None

    # A loop to attempt the API call multiple times if necessary
    i = 1
    while result is None:
        try:
            # Making a request to OpenAI's ChatCompletion API
            result = openai.ChatCompletion.create(
                model="gpt-4",  # Specifies the GPT model to use
                messages=[
                    {"role": "user", "content": prompt}  # The prompt or message for the GPT model
                ],
                temperature=0.5  # Sets the creativity of the response. Lower is more deterministic.
            )

        except openai.InvalidRequestError:
            # If there is an invalid request, store an error message
            result = "InvalidRequestError"
        except:
            # For other errors, wait for 1.5 seconds before retrying
            time.sleep(1.5)
            pass

    # Return the API response or error message
    return result



In [ ]:
transcript = "T: mobs are large crowds of people Usually mobs are not very happy. When they use the word mob they are angry or upset. Another word: gradually. It means that it will happen slowly or taking place. Moving along little by little. S: Like when babies learn to walk T: Yes like babies. They crawl and stand and eventually walk. T: Budge. It means that you gave in. You agree. You give in. T: Summarizes the meaning of the words above. T; We are going to use our inferring poster, which is on the pocket chart. We can use pictures or illustrations to infer. We infer by using our schema. The chart is color coded. T: S1 S1: They were inﬁghting inside the school. T: write I wander why the people behind the little girl were angry. Let’s begin. Hold that though and let us see if the author will answer our question. The title is the Story of Ruby Bridges. I can infer that the man facing the crowd or mob is trying to keep the people back. Writes this question on the chart. S2: he is ruby bridges dad. T: What makes you think that? T: I can infer the men facing the mob are trying to keep the mob back because they are spreading out their arms. Writes the inference on the chart as her inference. T\\: Begins to read the story. S3: I can infer that that is her family. T: How can you infer that? S;3 They have the same skin color. T: Continues to read. S:4: I think this is like Rosa Parks. T: Sounds like you are making a text to text to connection."
prompt = prompt + transcript

In [ ]:
result = send_to_api(prompt)
result

In [ ]:
result.choices[0].message['content']

Here are functions

In [ ]:
def chunk_and_send(pdf_path, prompt):
    loader = PyPDFLoader(file)
    docs = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 50000, chunk_overlap = 0)
    texts = text_splitter.split_documents(docs)

    directory = 'index_store'

    vector_index = FAISS.from_documents(texts, OpenAIEmbeddings())
    vector_index.save_local(directory)

    vector_index = FAISS.load_local('index_store', OpenAIEmbeddings())
    retriever = vector_index.as_retriever(search_type = "similarity", search_kwargs = {"k":6})
    qa_interface = RetrievalQA.from_chain_type(llm = ChatOpenAI(model_name = "gpt-4-turbo-preview"), chain_type = "stuff", retriever = retriever,
                                                   return_source_documents = True)

    response = qa_interface(prompt)

    return response['result']